# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to make sure that the team is efficiently monitoring and analyzing market data for AAPL to identify trends and predict market movements.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for AAPL using statistical modeling and machine learning", "context": "The goal is to identify trends and predict market movements for AAPL."}

> Entering new CrewAgentExecutor chain...
I need to gather real-time market data for AAPL in order to identify trends and predict market movements accurately.

Action: Read website content
Action Input: {"website_url": "https://finance.yahoo.com/quote/AAPL"} 

Apple Inc. (AAPL) Stock Price, News, Quot

Thought:
I need to ensure that the Data Analyst is continuously monitoring and analyzing the market data for AAPL using statistical modeling and machine learning to identify trends and predict market movements.

Action: Ask question to co-worker
Action Input: {"coworker": "Data Analyst", "question": "What statistical models and machine learning techniques are you using to analyze the market data for AAPL?", "context": "The goal is to identify trends and predict market movements for AAPL."}

> Entering new CrewAgentExecutor chain...
I should start by reading the website content for information on the statistical models and machine learning techniques being used to analyze market data for AAPL.

Action: Read website content
Action Input: {"website_url": "https://www.marketdataanalysis.com/aapl-statistical-models-ml-techniques"} 

I encountered an error while trying to use the tool. This was the error: HTTPSConnectionPool(host='www.marketdataanalysis.com', port=443): Max retries exceeded 

I need to delegate work to the Trading Strategy Developer to come up with potential trading strategies for AAPL based on the insights from the Data Analyst and the user-defined risk tolerance.

Action: Delegate work to co-worker
Action Input: {"coworker": "Trading Strategy Developer", "task": "Develop potential trading strategies for AAPL based on insights from Data Analyst and user-defined risk tolerance (Medium) with a focus on Day Trading preferences", "context": "Data Analyst is utilizing statistical models and machine learning techniques such as Random Forest Regressor, Logistic Regression model, Long Short-Term Memory model, and other AI-based techniques to analyze market data for AAPL. The focus is on using deep learning models and algorithmic trading to gain insights into AAPL's stock price movements."}


> Entering new CrewAgentExecutor chain...
I need to gather insights from the Data Analyst's findings on AAPL and combine them with my knowledge of trading strategies to develo

I need to delegate tasks and ask questions to ensure the best execution methods for trading AAPL are determined.

Action: Delegate work to co-worker
Action Input: {"coworker": "Trading Strategy Developer", "task": "Develop detailed execution plans for day trading AAPL", "context": "The average price target for AAPL is around $247.43, with a high estimate of $300 and a low estimate of $186. Recent trends in analyst recommendations show a positive shift, with multiple upgrades emphasizing growth potential. Technical analysis indicators such as moving averages, RSI, and Bollinger Bands should be utilized for entry and exit points. Stop-loss orders should also be incorporated for risk management."}

> Entering new CrewAgentExecutor chain...
I need to gather information on recent trends in analyst recommendations, technical analysis indicators, and stop-loss orders for day trading AAPL.

Action: Search the internet
Action Input: {"search_query": "recent trends in analyst recommendations for

Thought: I have gathered information on recent trends in analyst recommendations, technical analysis indicators, and stop-loss orders for day trading AAPL. I now need to develop detailed execution plans based on these insights.

Final Answer: 
After conducting thorough research on recent trends in analyst recommendations, technical analysis indicators, and stop-loss orders for day trading AAPL, I have developed the following detailed execution plan:

1. Analyst Recommendations:
- The average price target for AAPL is around $247.43, with a high estimate of $300 and a low estimate of $186.
- Recent trends in analyst recommendations show a positive shift, with multiple upgrades emphasizing growth potential.

2. Technical Analysis Indicators:
- Utilize moving averages, RSI, and Bollinger Bands for entry and exit points.
- Consider using additional indicators such as Ichimoku Cloud, MACD, and pivot points for comprehensive analysis.

3. Stop-Loss Orders:
- Implement stop-loss orders to mana

Final Answer: 
After reviewing the website content and conducting an internet search on AAPL's recent trends, analyst recommendations, and technical analysis indicators, I have gathered valuable insights to evaluate the risks associated with the proposed trading strategies.

1. Analyst Recommendations:
- The average price target for AAPL is around $247.43, with a high estimate of $300 and a low estimate of $186.
- Recent trends in analyst recommendations show a positive shift, with multiple upgrades emphasizing growth potential.

2. Technical Analysis Indicators:
- Utilize moving averages, RSI, and Bollinger Bands for entry and exit points.
- Consider using additional indicators such as Ichimoku Cloud, MACD, and pivot points for comprehensive analysis.

3. Stop-Loss Orders:
- Implement stop-loss orders to manage risk and limit potential losses.
- Set stop-loss levels based on technical analysis indicators and price action to ensure risk mitigation during day trading.

By integrating th

- Display the final result as Markdown.

In [16]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report for AAPL:

After evaluating the risks associated with the proposed trading strategies and execution plans for AAPL, the following potential risks have been identified along with suggested mitigation strategies:

1. Market Risk:
- The current market volatility poses a potential risk to the execution of the trading strategies for AAPL. To mitigate this risk, it is recommended to closely monitor market trends and adjust the trading strategies accordingly.

2. Stock-specific Risk:
- AAPL's stock price is subject to fluctuations based on various factors such as product launches, regulatory changes, and competition. To mitigate this risk, it is recommended to diversify the portfolio and not rely solely on AAPL for trading.

3. Technical Analysis Risk:
- Utilizing technical analysis indicators such as moving averages, RSI, and Bollinger Bands may provide valuable insights, but there is a risk of false signals and misinterpretation. To mitigate this risk, it is recommended to cross-verify signals with additional indicators and conduct thorough analysis before making trading decisions.

4. Stop-Loss Order Risk:
- While stop-loss orders are effective in managing risk, there is a risk of slippage during high volatility periods. To mitigate this risk, it is recommended to set stop-loss levels based on price action and adjust them regularly to align with market conditions.

Overall, by proactively monitoring market trends, diversifying the portfolio, conducting thorough technical analysis, and adjusting stop-loss levels accordingly, the risks associated with the proposed trading strategies for AAPL can be effectively mitigated.